In [281]:
import re, os, sys, shutil
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')

In [282]:
df = pd.read_csv('clean_all.csv')
df.head(5)

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type
0,PG0,NaN,NaN,NaN,NaN,NaN,NaN,set(),Text
1,PG10000,The Magna Carta,Anonymous,NaN,NaN,['en'],167.0,"{'Magna Carta', 'Constitutional history -- Eng...",Text
2,PG10001,Apocolocyntosis,"Seneca, Lucius Annaeus",NaN,65.0,['en'],212.0,"{'Claudius, Emperor of Rome, 10 B.C.-54 A.D. -...",Text
3,PG10002,The House on the Borderland,"Hodgson, William Hope",1877.0,1918.0,['en'],599.0,{'Science fiction'},Text
4,PG10003,"My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King",1833.0,1923.0,['en'],16.0,"{'France -- History -- Third Republic, 1870-19...",Text


In [283]:
df.shape

(59220, 9)

In [284]:
# remove nulls
df = df.dropna(how='any',axis=0) 
df.shape

(42769, 9)

In [285]:
# remove none english
df = df[df.language == '[\'en\']']
df.shape

(33752, 9)

In [286]:
df.language.value_counts()

['en']    33752
Name: language, dtype: int64

In [287]:
# remove none text
df = df[df.type != 'text']
df.shape

(33752, 9)

In [288]:
# sort by downloads
df.sort_values("downloads", inplace = True, ascending=False) 
df.head(5)

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type
3810,PG1342,Pride and Prejudice,"Austen, Jane",1775.0,1817.0,['en'],47860.0,"{'England -- Fiction', 'Young women -- Fiction...",Text
57551,PG84,"Frankenstein; Or, The Modern Prometheus","Shelley, Mary Wollstonecraft",1797.0,1851.0,['en'],25433.0,"{'Science fiction', 'Monsters -- Fiction', ""Fr...",Text
17142,PG2542,A Doll's House : a play,"Ibsen, Henrik",1828.0,1906.0,['en'],21753.0,"{'Man-woman relationships -- Drama', 'Wives --...",Text
59106,PG98,A Tale of Two Cities,"Dickens, Charles",1812.0,1870.0,['en'],21525.0,{'Paris (France) -- History -- 1789-1799 -- Fi...,Text
899,PG1080,A Modest Proposal: For preventing the children...,"Swift, Jonathan",1667.0,1745.0,['en'],21140.0,"{'Religious satire, English', 'Ireland -- Poli...",Text


In [289]:
# query publication year from wikipedia
import wikipedia, re, string, sys
def wiki(title):
    result = wikipedia.summary(title)
    year = re.search("publish.*([0-9]{4}s*)", result, flags=0)
    if year is not None:
        year = year.group(1)
    return year
wiki("Frankenstein; Or, The Modern Prometheus")

'1823'

In [366]:
df_curr = df[1200:5000]
df_curr.shape

(3800, 10)

In [ ]:
df_curr['publicationyear'] = None
for title in df_curr['title']:
    try:
        publicationyear = wiki(title)
    except:
        publicationyear = None
        pass
    df_curr.loc[df_curr['title'] == title, ['publicationyear']] = publicationyear

In [347]:
df_curr.head(5)

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,timeperiod,publicationyear
299,PG1026,The Diary of a Nobody,"Grossmith, Weedon",1852.0,1919.0,['en'],604.0,"{'England -- Fiction', 'Suburban life -- Ficti...",Text,None,None
54712,PG5946,"The History of Don Quixote, Volume 2, Complete","Cervantes Saavedra, Miguel de",1547.0,1616.0,['en'],604.0,{'Spain -- Social life and customs -- 16th cen...,Text,None,None
7975,PG1717,What's Wrong with the World,"Chesterton, G. K. (Gilbert Keith)",1874.0,1936.0,['en'],604.0,{'Social problems'},Text,None,None
8075,PG1726,Theaetetus,Plato,-428.0,-348.0,['en'],604.0,"{'Classical literature', 'Knowledge, Theory of...",Text,None,None
18979,PG27083,The Subjection of Women,"Mill, John Stuart",1806.0,1873.0,['en'],603.0,"{""Women's rights"", 'Women -- Social and moral ...",Text,None,1861


In [348]:
# remove nulls in publicationyear
df_curr = df_curr.dropna(subset=['publicationyear'],axis=0) 
df_curr.shape

(218, 11)

In [349]:
# remove xxxxs format years
df_curr['publicationyear'] = df_curr['publicationyear'].astype(str)
searchfor = ['s']
df_curr = df_curr[~df_curr['publicationyear'].str.contains('|'.join(searchfor))]
df_curr.shape

(209, 11)

In [350]:
# add age column (age = publicationyear - authoryearofbirth)
df_curr['publicationyear'] = df_curr['publicationyear'].astype('int')

In [351]:
df_curr['age'] = df_curr['publicationyear'] - df_curr['authoryearofbirth']
df_curr.head(5)

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,timeperiod,publicationyear,age
18979,PG27083,The Subjection of Women,"Mill, John Stuart",1806.0,1873.0,['en'],603.0,"{""Women's rights"", 'Women -- Social and moral ...",Text,None,1861,55.0
54475,PG59031,"Journal 02, 1850-September 15, 1851: The Writi...","Thoreau, Henry David",1817.0,1862.0,['en'],596.0,"{'Thoreau, Henry David, 1817-1862 -- Diaries',...",Text,None,1864,47.0
57240,PG821,Dombey and Son,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Fathers and daughters -- Fiction', 'Domestic...",Text,None,1848,36.0
58872,PG968,Martin Chuzzlewit,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Grandfathers -- Fiction', 'Bildungsromans', ...",Text,None,1842,30.0
20613,PG28554,Beyond Lies the Wub,"Dick, Philip K.",1928.0,1982.0,['en'],593.0,"{'Science fiction', 'Short stories'}",Text,None,1977,49.0


In [352]:
# age category
# 18-24    0
# 25-34    1
# 35-49    2
# 50-64    3
# 65+    4
age_lower_bounds = [18, 25, 35, 50, 65, 100]
def get_age_category(year):
    year = int(year)
    if (year < 18):
        return None
    else:
        for i in range(0, 5):
            if (year >= age_lower_bounds[i] and year < age_lower_bounds[i+1]):
                return i

In [353]:
df_curr['agecategory'] = None
i = 0
for year in df_curr['age']:
    try:
        age_category = get_age_category(year)
    except:
        age_category = None
        pass
    df_curr.loc[df_curr['age'] == year, ['agecategory']] = age_category
df_curr.head(10)

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,timeperiod,publicationyear,age,agecategory
18979,PG27083,The Subjection of Women,"Mill, John Stuart",1806.0,1873.0,['en'],603.0,"{""Women's rights"", 'Women -- Social and moral ...",Text,None,1861,55.0,3
54475,PG59031,"Journal 02, 1850-September 15, 1851: The Writi...","Thoreau, Henry David",1817.0,1862.0,['en'],596.0,"{'Thoreau, Henry David, 1817-1862 -- Diaries',...",Text,None,1864,47.0,2
57240,PG821,Dombey and Son,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Fathers and daughters -- Fiction', 'Domestic...",Text,None,1848,36.0,2
58872,PG968,Martin Chuzzlewit,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Grandfathers -- Fiction', 'Bildungsromans', ...",Text,None,1842,30.0,1
20613,PG28554,Beyond Lies the Wub,"Dick, Philip K.",1928.0,1982.0,['en'],593.0,"{'Science fiction', 'Short stories'}",Text,None,1977,49.0,2
577,PG1051,Sartor Resartus: The Life and Opinions of Herr...,"Carlyle, Thomas",1795.0,1881.0,['en'],591.0,"{'Germany -- Fiction', 'Clothing and dress -- ...",Text,None,1834,39.0,2
12775,PG2149,The Works of Edgar Allan Poe — Volume 3,"Poe, Edgar Allan",1809.0,1849.0,['en'],586.0,"{'Horror tales, American', 'American fiction -...",Text,None,1829,20.0,0
26050,PG33447,The Pencil of Nature,"Talbot, William Henry Fox",1800.0,1877.0,['en'],585.0,{'Photography -- Early works to 1850'},Text,None,1846,46.0,2
6276,PG1564,Boswell's Life of Johnson: Abridged and edited...,"Boswell, James",1740.0,1795.0,['en'],574.0,"{'Authors, English -- 18th century -- Biograph...",Text,None,1763,23.0,0
51663,PG564,The Mystery of Edwin Drood,"Dickens, Charles",1812.0,1870.0,['en'],573.0,"{'England -- Fiction', 'Missing persons -- Fic...",Text,None,1870,58.0,3


In [354]:
# remove nulls in age categories
df_curr = df_curr.dropna(subset=['agecategory'],axis=0) 
df_curr.shape

(179, 13)

In [355]:
# get time period
# 1751-1800 0
# 1801-1820 1
# 1821-1840 2
# 1841-1860 3
# 1861-1880 4
# 1881-1900 5
# 1901-1920 6
time_period_lower_bounds = [1751, 1801, 1821, 1841, 1861, 1881, 1901, 2000]
def get_time_period_category(year):
    year = int(year)

    if (year < 1751 or year > 1920):
        return None
    else:
        for i in range(0, 6):
            if (year >= time_period_lower_bounds[i] and year < time_period_lower_bounds[i+1]):
                return i

In [356]:
df['timeperiod'] = None
for year in df_curr['publicationyear']:
    try:
        category = get_time_period_category(year)
    except:
        category = None
        pass
    df_curr.loc[df_curr['publicationyear'] == year, 'timeperiod'] = category
df_curr.head(6)

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,timeperiod,publicationyear,age,agecategory
18979,PG27083,The Subjection of Women,"Mill, John Stuart",1806.0,1873.0,['en'],603.0,"{""Women's rights"", 'Women -- Social and moral ...",Text,4,1861,55.0,3
54475,PG59031,"Journal 02, 1850-September 15, 1851: The Writi...","Thoreau, Henry David",1817.0,1862.0,['en'],596.0,"{'Thoreau, Henry David, 1817-1862 -- Diaries',...",Text,4,1864,47.0,2
57240,PG821,Dombey and Son,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Fathers and daughters -- Fiction', 'Domestic...",Text,3,1848,36.0,2
58872,PG968,Martin Chuzzlewit,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Grandfathers -- Fiction', 'Bildungsromans', ...",Text,3,1842,30.0,1
20613,PG28554,Beyond Lies the Wub,"Dick, Philip K.",1928.0,1982.0,['en'],593.0,"{'Science fiction', 'Short stories'}",Text,None,1977,49.0,2
577,PG1051,Sartor Resartus: The Life and Opinions of Herr...,"Carlyle, Thomas",1795.0,1881.0,['en'],591.0,"{'Germany -- Fiction', 'Clothing and dress -- ...",Text,2,1834,39.0,2


In [357]:
# remove nulls in publication year
df_curr = df_curr.dropna(how='any',axis=0) 
df_curr.shape

(109, 13)

In [ ]:
# remove = []
# for index in remove:
#     index+=str(i)

In [47]:
# # manually remove none fiction
# df_curr = df_curr[~df_curr['txt'].isin(remove)]
# df_curr.shape

(1029, 3)

In [358]:
for txt in df_curr['id']:
    try:
        os.rename(os.path.join("text", str(txt + "_text.txt")), os.path.join("clean", str(txt + "_text.txt")))
    except FileNotFoundError as err:
        df_curr = df_curr[~df_curr['id'].isin([txt])]

In [359]:
df_curr['id'] = df_curr['id'].astype(str) + '_text.txt'
df_curr.head()

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,timeperiod,publicationyear,age,agecategory
18979,PG27083_text.txt,The Subjection of Women,"Mill, John Stuart",1806.0,1873.0,['en'],603.0,"{""Women's rights"", 'Women -- Social and moral ...",Text,4,1861,55.0,3
54475,PG59031_text.txt,"Journal 02, 1850-September 15, 1851: The Writi...","Thoreau, Henry David",1817.0,1862.0,['en'],596.0,"{'Thoreau, Henry David, 1817-1862 -- Diaries',...",Text,4,1864,47.0,2
57240,PG821_text.txt,Dombey and Son,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Fathers and daughters -- Fiction', 'Domestic...",Text,3,1848,36.0,2
58872,PG968_text.txt,Martin Chuzzlewit,"Dickens, Charles",1812.0,1870.0,['en'],595.0,"{'Grandfathers -- Fiction', 'Bildungsromans', ...",Text,3,1842,30.0,1
577,PG1051_text.txt,Sartor Resartus: The Life and Opinions of Herr...,"Carlyle, Thomas",1795.0,1881.0,['en'],591.0,"{'Germany -- Fiction', 'Clothing and dress -- ...",Text,2,1834,39.0,2


In [360]:
df_curr.shape

(105, 13)

In [361]:
df_curr.timeperiod.value_counts()

5    34
3    19
4    17
2    17
0    14
1     4
Name: timeperiod, dtype: int64

In [362]:
df_curr.agecategory.value_counts()

2    30
1    26
3    24
4    15
0    10
Name: agecategory, dtype: int64

In [363]:
with open('clean_verbose.csv', 'a') as f:
    df_curr.to_csv(f, header=False, index=True)
df_curr.columns.tolist()

['id',
 'title',
 'author',
 'authoryearofbirth',
 'authoryearofdeath',
 'language',
 'downloads',
 'subjects',
 'type',
 'timeperiod',
 'publicationyear',
 'age',
 'agecategory']

In [364]:
df_curr.drop(['title',
 'author',
 'authoryearofbirth',
 'authoryearofdeath',
 'language',
 'downloads',
 'subjects',
 'type',
 'publicationyear',
 'age'], axis = 1, inplace = True)
df_curr.head()

,id,timeperiod,agecategory
18979,PG27083_text.txt,4,3
54475,PG59031_text.txt,4,2
57240,PG821_text.txt,3,2
58872,PG968_text.txt,3,1
577,PG1051_text.txt,2,2


In [365]:
with open('clean.csv', 'a') as f:
    df_curr.to_csv(f, header=False, index=False)